In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_data=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv",encoding="latin1")
test_data=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv",encoding="latin1")

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
Id=test_data.id

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
train_data.describe()

# Data Cleaning 

In [ ]:
import re 
train_data['text'] = [re.sub("[^a-zA-Z]"," ",text).lower() for text in train_data['text']]
test_data['text'] = [re.sub("[^a-zA-Z]"," ",text).lower() for text in test_data['text']]

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
from textblob import TextBlob

def polarity_check_label(dataframe):
    polarity_list = []
    
    for text in dataframe['text']:
        polarity_point = TextBlob(text).sentiment.polarity
        if polarity_point < 0 : polarity_state= 'Negative'
        elif polarity_point == 0 : polarity_state = 'Neutral'
        else : polarity_state = 'Positive'
        polarity_list.append(polarity_state)
        
    dataframe['polarity_state'] = polarity_list
    
polarity_check_label(train_data)
polarity_check_label(test_data)

# Stopping Irrelavnt Words

In [ ]:
import nltk # natural language tool kit for word_tokenize ...
nltk.download("stopwords")      # stopwords is download in corpus directory
from nltk.corpus import stopwords  # import stopwords

In [ ]:
train_data['text'] = [nltk.word_tokenize(text) for text in train_data['text']]
test_data['text'] = [nltk.word_tokenize(text) for text in test_data['text']]

train_data.text.head()

In [ ]:
test_data.text.head()

In [ ]:
def lemma_and_join(dataframe):
    lemma = nltk.WordNetLemmatizer()
    text_list = []
    for text in dataframe['text']:
        text = [ word for word in text if not word in set(stopwords.words("english"))]
        text = [lemma.lemmatize(word) for word in text]
        text = " ".join(text)
        text_list.append(text)
    return text_list

In [ ]:
train_data['text'] = lemma_and_join(train_data)
test_data['text'] = lemma_and_join(test_data)
train_data.head()

In [ ]:
raw_loc = train_data.location.value_counts()
top_loc_disaster = list(raw_loc[raw_loc>=10].index)
top_only_disaster = train_data[train_data.location.isin(top_loc_disaster)]

top_location = top_only_disaster.groupby('location')['target'].mean().sort_values(ascending=False)
sns.barplot(x=top_location.index, y=top_location)
plt.xticks(rotation=90)
plt.show()

In [ ]:
len(set(train_data['location'])) 

In [ ]:
def clean_location(x):
    if x == 'None':
        return 'None'
    elif x == 'Earth' or x =='Worldwide' or x == 'Everywhere':
        return 'World'
    elif 'New York' in x or 'NYC' in x:
        return 'New York'    
    elif 'London' in x:
        return 'London'
    elif 'Mumbai' in x:
        return 'Mumbai'
    elif 'Washington' in x and 'D' in x and 'C' in x:
        return 'Washington DC'
    elif 'San Francisco' in x:
        return 'San Francisco'
    elif 'Los Angeles' in x:
        return 'Los Angeles'
    elif 'Seattle' in x:
        return 'Seattle'
    elif 'Chicago' in x:
        return 'Chicago'
    elif 'Toronto' in x:
        return 'Toronto'
    elif 'Sacramento' in x:
        return 'Sacramento'
    elif 'Atlanta' in x:
        return 'Atlanta'
    elif 'California' in x:
        return 'California'
    elif 'Florida' in x:
        return 'Florida'
    elif 'Texas' in x:
        return 'Texas'
    elif 'United States' in x or 'USA' in x:
        return 'USA'
    elif 'United Kingdom' in x or 'UK' in x or 'Britain' in x:
        return 'UK'
    elif 'Canada' in x:
        return 'Canada'
    elif 'India' in x:
        return 'India'
    elif 'Kenya' in x:
        return 'Kenya'
    elif 'Nigeria' in x:
        return 'Nigeria'
    elif 'Australia' in x:
        return 'Australia'
    elif 'Indonesia' in x:
        return 'Indonesia'
    elif x in top_location:
        return x
    else: 
        return 'Others'
    
train_data['location'] = train_data['location'].apply(lambda x: clean_location(str(x)))
test_data['location'] = test_data['location'].apply(lambda x: clean_location(str(x)))

In [ ]:
top_location = train_data.groupby('location')['target'].mean().sort_values(ascending=False)
plt.figure(figsize=(14,6))
sns.barplot(x=top_location.index, y=top_location)
plt.xticks(rotation=90)
plt.show()

In [ ]:
len(set(train_data['location'])) 

In [ ]:
len(set(test_data['location'])) 

In [ ]:
freq_df = train_data['text'].apply(lambda x:pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
freq_df.columns = ['words', 'frequences']
freq_df.sort_values('frequences',ascending=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
max_features=5000

count_vectorizer = CountVectorizer(max_features=max_features)

sparce_matrix_train=count_vectorizer.fit_transform(train_data['text'])
sparce_matrix_test=count_vectorizer.fit_transform(test_data['text'])

print("{} most used words: {} ".format(max_features,count_vectorizer.get_feature_names()))

In [ ]:
from textblob import TextBlob
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

In [ ]:
import category_encoders as ce

features = ['keyword', 'location']
encoder = ce.TargetEncoder(cols=features)
encoder.fit(train_data[features],train_data['target'])

train_data = train_data.join(encoder.transform(train_data[features]).add_suffix('_target'))
test_data = test_data.join(encoder.transform(test_data[features]).add_suffix('_target'))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec_text = TfidfVectorizer(min_df = 10, ngram_range = (1,2), stop_words='english') 
text_vec = vec_text.fit_transform(train_data['text'])
text_vec_test = vec_text.transform(test_data['text'])
X_train_text = pd.DataFrame(text_vec.toarray(), columns=vec_text.get_feature_names())
X_test_text = pd.DataFrame(text_vec_test.toarray(), columns=vec_text.get_feature_names())

In [ ]:
train_data = train_data.join(X_train_text, rsuffix='_text')
test_data=test_data.join(X_test_text,rsuffix='_text')

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score,precision_score,recall_score


In [ ]:
y = train_data.target
x = train_data.drop(columns = ['id','keyword', 'location', 'text','polarity_state','target'])

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

# Model Building and Prediction

In [ ]:
f1_score_list=[]
train_accuracy_list=[]
test_accuracy_list = []
classifier_list = []
def fit_and_predict(model,x_train,x_test,y_train,y_test):
    
    classifier = model
    classifier.fit(x_train,y_train)
    y_pred = classifier.predict(x_test)
    cmatrix = confusion_matrix(y_test,y_pred)
    
    
    f,ax = plt.subplots(figsize=(3,3))
    sns.heatmap(cmatrix,annot=True,linewidths=0.5,cbar=False,linecolor="red",fmt='.0f',ax=ax)
    plt.xlabel("y_predict")
    plt.ylabel("y_true")
    ax.set(title=str(classifier))
    plt.show()
    
    
    f1score = f1_score(y_test,y_pred,average='weighted')
    train_accuracy = round(classifier.score(x_train,y_train)*100)
    test_accuracy =  round(accuracy_score(y_test,y_pred)*100)
    
    classifier_list.append(str(classifier))
    train_accuracy_list.append(str(train_accuracy))
    test_accuracy_list.append(str(test_accuracy))
    f1_score_list.append(str(round(f1score*100)))
    
    
    print(classification_report(y_test,y_pred))
    print('Accuracy of classifier on training set:{}%'.format(train_accuracy))
    print('-'*50)
    print('Accuracy of classifier on test set:{}%' .format(test_accuracy))


In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
random_state = 29
models=[
        LogisticRegression(random_state=29),
        SVC(random_state=random_state),
        MultinomialNB(),
        DecisionTreeClassifier(random_state = 29),
        KNeighborsClassifier(),
        RandomForestClassifier(random_state=29),
       ]

In [ ]:
for model in models:
    fit_and_predict(model,x_train,x_test,y_train,y_test)

In [ ]:
columns=x_train.columns
columns

In [ ]:
test = test_data.reindex(columns = columns, fill_value=0)

In [ ]:
test.shape

In [ ]:
test.columns

In [ ]:
pred = model.predict(test)
pred.shape

In [ ]:
#submission = pd.DataFrame(data=pred,columns=['Traget'])

In [ ]:
submission = pd.DataFrame({"id":Id, "target": pred})
submission.to_csv("submission.csv", index=False)